In [1]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np

In [2]:
challenger = pd.read_csv("challenger_match_data.csv", index_col=0)


In [3]:
challenger_players = challenger.groupby(["gameId"]).size().reset_index(name = 'n_challengers')
challenger_players

,gameId,n_challengers
0,4083343388,1
1,4083362752,1
2,4083389656,1
3,4083401236,1
4,4083456764,1
...,...,...
12646,4261001693,4
12647,4261012162,4
12648,4261013083,1
12649,4261020773,7


In [4]:
match = pd.read_csv("match_data_version1.csv", index_col=0)

In [5]:
match_trim = match.loc[:,["gameId", "gameMode", "gameType", "seasonId"]].drop_duplicates()

In [6]:
match_trim

,gameId,gameMode,gameType,seasonId
0,4.247263e+09,CLASSIC,MATCHED_GAME,13.0
1,4.247156e+09,CLASSIC,MATCHED_GAME,13.0
2,4.243963e+09,CLASSIC,MATCHED_GAME,13.0
3,4.241678e+09,CLASSIC,MATCHED_GAME,13.0
4,4.241539e+09,CLASSIC,MATCHED_GAME,13.0
...,...,...,...,...
0,4.088497e+09,URF,MATCHED_GAME,13.0
0,4.130964e+09,CLASSIC,MATCHED_GAME,13.0
0,4.119954e+09,CLASSIC,MATCHED_GAME,13.0
0,4.130440e+09,URF,MATCHED_GAME,13.0


In [7]:
winner = pd.read_csv("match_winner_data_version1.csv", index_col = 0)
winner.head()
winner_trim = winner[["gameId", "firstBlood", "firstTower", "towerKills"]].drop_duplicates()
winner_named = winner_trim.rename(columns = {"firstBlood": "winnerFirstBlood", "firstTower": "winnerFirstTower", "towerKills": "winnerTowerKills"})
winner_named

,gameId,winnerFirstBlood,winnerFirstTower,winnerTowerKills
0,4.247263e+09,False,True,9
1,4.247156e+09,False,False,4
2,4.243963e+09,True,True,5
3,4.241678e+09,True,True,6
4,4.241539e+09,True,True,11
...,...,...,...,...
108824,4.088497e+09,False,True,9
108825,4.130964e+09,True,False,5
108826,4.119954e+09,False,False,6
108827,4.130440e+09,True,True,10


In [8]:
loser = pd.read_csv("match_loser_data_version1.csv", index_col = 0)
loser.head()
loser_trim = loser[["gameId", "firstBlood", "firstTower", "towerKills"]].drop_duplicates()
loser_named = loser_trim.rename(columns = {"firstBlood": "loserFirstBlood", "firstTower": "loserFirstTower", "towerKills": "loserTowerKills"})
loser_named

,gameId,loserFirstBlood,loserFirstTower,loserTowerKills
0,4.247263e+09,True,False,0
1,4.247156e+09,True,True,2
2,4.243963e+09,False,False,0
3,4.241678e+09,False,False,2
4,4.241539e+09,False,False,6
...,...,...,...,...
108824,4.088497e+09,True,False,3
108825,4.130964e+09,False,True,3
108826,4.119954e+09,True,True,6
108827,4.130440e+09,False,False,5


In [9]:
match_merged = match_trim.merge(winner_named, on = "gameId", how="left")\
    .merge(loser_named, on="gameId", how="left")\
    .merge(challenger_players, on="gameId", how="left")
match_merged

,gameId,gameMode,gameType,seasonId,winnerFirstBlood,winnerFirstTower,winnerTowerKills,loserFirstBlood,loserFirstTower,loserTowerKills,n_challengers
0,4.247263e+09,CLASSIC,MATCHED_GAME,13.0,False,True,9,True,False,0,10.0
1,4.247156e+09,CLASSIC,MATCHED_GAME,13.0,False,False,4,True,True,2,10.0
2,4.243963e+09,CLASSIC,MATCHED_GAME,13.0,True,True,5,False,False,0,10.0
3,4.241678e+09,CLASSIC,MATCHED_GAME,13.0,True,True,6,False,False,2,8.0
4,4.241539e+09,CLASSIC,MATCHED_GAME,13.0,True,True,11,False,False,6,10.0
...,...,...,...,...,...,...,...,...,...,...,...
108768,4.088497e+09,URF,MATCHED_GAME,13.0,False,True,9,True,False,3,NaN
108769,4.130964e+09,CLASSIC,MATCHED_GAME,13.0,True,False,5,False,True,3,NaN
108770,4.119954e+09,CLASSIC,MATCHED_GAME,13.0,False,False,6,True,True,6,NaN
108771,4.130440e+09,URF,MATCHED_GAME,13.0,True,True,10,False,False,5,NaN


In [10]:
# Replace NA challengers with 0
match_merged['n_challengers'] = match_merged['n_challengers'].fillna(0)

# Convert Booleans to 1 or 0
match_merged['winnerFirstBlood'] = match_merged['winnerFirstBlood']*1
match_merged['winnerFirstTower'] = match_merged['winnerFirstTower']*1
match_merged['loserFirstBlood'] = match_merged['loserFirstBlood']*1
match_merged['loserFirstTower'] = match_merged['loserFirstTower']*1

# Convert GameID to Integer
match_merged["gameId"].astype(int)

match_merged = match_merged.reset_index(drop=True).set_index(["gameId"])

match_merged.head()

,gameMode,gameType,seasonId,winnerFirstBlood,winnerFirstTower,winnerTowerKills,loserFirstBlood,loserFirstTower,loserTowerKills,n_challengers
gameId,,,,,,,,,,
4.247263e+09,CLASSIC,MATCHED_GAME,13.0,0,1,9,1,0,0,10.0
4.247156e+09,CLASSIC,MATCHED_GAME,13.0,0,0,4,1,1,2,10.0
4.243963e+09,CLASSIC,MATCHED_GAME,13.0,1,1,5,0,0,0,10.0
4.241678e+09,CLASSIC,MATCHED_GAME,13.0,1,1,6,0,0,2,8.0
4.241539e+09,CLASSIC,MATCHED_GAME,13.0,1,1,11,0,0,6,10.0


### Connect to Database

In [11]:
rds_connection_string = "postgres:postgres@localhost:5432/league_of_legends"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Copy Local File to Database

In [12]:
engine.table_names()

['match_features']

In [13]:
match_merged.to_sql(name='match_features', con=engine, if_exists='append', index=True)